<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Дальнейшие-действия" data-toc-modified-id="Дальнейшие-действия-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Дальнейшие действия</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

<b>Задача: обучить модель классифицировать комментарии на позитивные и негативные и построить модель со значением метрики качества *F1* не меньше 0.75. </b>:

## Подготовка

Сначала отдельно импортируем все необходимые библиотеки и функции:

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stop_words = stopwords.words('english')

pd.set_option('display.max_colwidth', 1000)
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Далее откроем и изучим файл с данными:

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0


Пропусков нет, но избавимся от бесполезного столбца 'Unnamed: 0'. Сохраним значение переменной гиперпараметра random_state:

In [3]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
rand_state = 12345

Теперь напишем большой класс, реализующий сразу все этапы предобработки и классификации комментариев:

In [4]:
class comments_classification():
    
    def __init__(self,
                 models_and_params: list,
                 score: str,
                 solvers: list,
                 stop_words,
                 start_frame,
                 target_column,
                 data_column):
        # Инициализация всех переданных параметров
        self.models_and_params = models_and_params
        self.score = score
        self.solvers = solvers
        self.stop_words = stop_words
        self.start_frame = start_frame
        self.target_column = target_column
        self.data_column = data_column
        self.lemm_corpus = None
        self.splited_data = None
        self.vect = None
        self.max_score = -1
        self.best_model = None
        
        # preprocessing
        self.__text_clearning()
        print("Первый этап пройден")
        self.__lemmatisation()
        print("Второй этап пройден")
        self.__splitter()
        print("Третий этап пройден")
        self.__vectorisation()
        print("Подготовка завершена")

        
    def __splitter(self):
        """ Функция, разделяющая обработанные данные на тренировочную,
        валидационную и тестовую
        """

        presplited_data = train_test_split(
            self.lemm_corpus, self.start_frame[self.target_column], test_size = 0.2, random_state = rand_state)
        splited_data_w_val = train_test_split(
            presplited_data[1], presplited_data[3], test_size = 0.5, random_state = rand_state)
        self.splited_data = [presplited_data[0], splited_data_w_val[0], splited_data_w_val[1],
                             presplited_data[2], splited_data_w_val[2], splited_data_w_val[3]]

        
    def __lemmatisation(self):
        """Функция, отвечающая за лемматизацию слов корпуса
        """
        # Инициализация лемматизатора
        lemmatizer = WordNetLemmatizer()
        # Лемматизация корпуса
        self.lemm_corpus = self.corpus.apply(
            lambda sentence: " ".join([lemmatizer.lemmatize(word, "n") for word in nltk.word_tokenize(sentence)]))

        
    def __text_clearning(self):
        """ Функция, отвечающая за очистку корпуса от лишних символов
        """
        # Выделение корпуса для дальнейшего анализа
        corpus = self.start_frame[self.data_column]
        # Очистка корпуса от лишних символов
        self.corpus = corpus.apply(lambda sentence: re.sub(r'[^a-zA-Z]', ' ', sentence))
        
    def __vectorisation(self):
        """Функция, отвечающая за векторизацию корпуса
        """
        # Создание словаря со словарями, которые хранят в себе векторизованные данные от разных векторизаторов 
        self.vect = {str(i()):{} for i in self.solvers}
        # Векторизация данных разными методами
        for vectorizer in self.solvers:
            # Инициализация векторизатора и установка стоп-слов
            vector = vectorizer(stop_words = self.stop_words)
            # Обучение и трансформация на обучающей выборке
            self.vect[str(vectorizer())]['train_data'] = vector.fit_transform(self.splited_data[0])
            self.vect[str(vectorizer())]['train_target'] = self.splited_data[3]
            # Трансформация тестовой выборки
            self.vect[str(vectorizer())]['test_data'] = vector.transform(self.splited_data[1])
            self.vect[str(vectorizer())]['test_target'] = self.splited_data[4]
            # Трансформация валидационной выборки
            self.vect[str(vectorizer())]['valid_data'] = vector.transform(self.splited_data[2])
            self.vect[str(vectorizer())]['valid_target'] = self.splited_data[5]
            
            
    def fit(self):
        """Тренируем все переданные модели
        """
        # Инициализация словаря
        self.black_boxes = {str(name):{} for name,_ in self.models_and_params}
        # Перебор всех моделей
        for model, params in tqdm(self.models_and_params):
            # Перебор всех векторизаторов
            for vectorizer, data in tqdm(self.vect.items(), desc = str(model)):
                # Инициализация внутреннего словаря
                self.black_boxes[str(model)][str(vectorizer)] = {}
                # Инициализация грида
                self.black_boxes[str(model)][str(vectorizer)]["grid_object"] = GridSearchCV(
                    model, params, cv=3, scoring=self.score)
                # Тренировка грида
                self.black_boxes[str(model)][str(vectorizer)]["grid_object"].fit(
                    self.vect[str(vectorizer)]['train_data'], self.vect[str(vectorizer)]['train_target'])
                # Сохранение лучшей модели
                self.black_boxes[str(model)][str(vectorizer)]["best_model"] = self.black_boxes[
                    str(model)][str(vectorizer)]["grid_object"].best_estimator_
                # Сохранение лучшего скора на разных выборках
                self.black_boxes[str(model)][str(vectorizer)]["best_score_train"] = self.black_boxes[
                    str(model)][str(vectorizer)]["grid_object"].best_score_
                self.black_boxes[str(model)][str(vectorizer)]["best_score_valid"] = f1_score(
                    self.vect[str(vectorizer)]['valid_target'], self.black_boxes[
                        str(model)][str(vectorizer)]["best_model"].predict(self.vect[str(vectorizer)]['valid_data']))
                self.black_boxes[str(model)][str(vectorizer)]["best_score_test"] = f1_score(
                    self.vect[str(vectorizer)]['test_target'],self.black_boxes[
                        str(model)][str(vectorizer)]["best_model"].predict(self.vect[str(vectorizer)]['test_data']))
                # Поиск максимального скора на валидационной выборке
                if self.black_boxes[str(model)][str(vectorizer)]["best_score_valid"] > self.max_score:
                    self.max_score = self.black_boxes[str(model)][str(vectorizer)]["best_score_test"]
                    self.best_model = self.black_boxes[str(model)][str(vectorizer)]["best_model"]
                    
        return {"max_score":self.max_score, "best_model":self.best_model}
    
    def get_info(self):
        """ Функция, возвращающая всю собранную информацию об обучении
        """
        return self.black_boxes

После этого можно сразу приступить к обучению моделей.

## Обучение

Определим параметры для модели и затем инициализируем класс:

In [5]:
%%time

params_logistic = {"max_iter": [1000, 2000, 100]}
params_random_forest = {"n_estimators": [40, 200, 20],"max_depth": [2, 10]}

comments_class = comments_classification(
    [(LogisticRegression(random_state = rand_state, class_weight='balanced'), params_logistic),
     (RandomForestClassifier(random_state = rand_state, class_weight='balanced'), params_random_forest)],
    'f1', [TfidfVectorizer, CountVectorizer], stop_words, df, 'toxic', 'text')

Первый этап пройден
Второй этап пройден
Третий этап пройден
Подготовка завершена
CPU times: user 1min 37s, sys: 424 ms, total: 1min 37s
Wall time: 1min 37s


Найдём наилучшую модель и метрику качества F1 для неё, используя специальную функцию класса, определённого выше:

In [6]:
%%time

comments_class.fit()

  0%|          | 0/2 [00:00<?, ?it/s]
LogisticRegression(class_weight='balanced', random_state=12345):   0%|          | 0/2 [00:00<?, ?it/s]
LogisticRegression(class_weight='balanced', random_state=12345):  50%|█████     | 1/2 [07:22<07:22, 442.41s/it]
LogisticRegression(class_weight='balanced', random_state=12345): 100%|██████████| 2/2 [33:56<00:00, 1018.36s/it]
 50%|█████     | 1/2 [33:56<33:56, 2036.72s/it]
RandomForestClassifier(class_weight='balanced', random_state=12345):   0%|          | 0/2 [00:00<?, ?it/s]
RandomForestClassifier(class_weight='balanced', random_state=12345):  50%|█████     | 1/2 [02:47<02:47, 167.40s/it]
RandomForestClassifier(class_weight='balanced', random_state=12345): 100%|██████████| 2/2 [05:31<00:00, 165.79s/it]
100%|██████████| 2/2 [39:28<00:00, 1184.15s/it]

CPU times: user 20min 44s, sys: 18min 41s, total: 39min 26s
Wall time: 39min 28s


{'max_score': 0.763172804532578,
 'best_model': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=12345)}

Посмотрим подробнее всю собранную информацию об обучении:

In [7]:
comments_class.get_info()

{"LogisticRegression(class_weight='balanced', random_state=12345)": {'TfidfVectorizer()': {'grid_object': GridSearchCV(cv=3,
                estimator=LogisticRegression(class_weight='balanced',
                                             random_state=12345),
                param_grid={'max_iter': [1000, 2000, 100]}, scoring='f1'),
   'best_model': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=12345),
   'best_score_train': 0.7452635998983105,
   'best_score_valid': 0.7466954410574588,
   'best_score_test': 0.7514200703272925},
  'CountVectorizer()': {'grid_object': GridSearchCV(cv=3,
                estimator=LogisticRegression(class_weight='balanced',
                                             random_state=12345),
                param_grid={'max_iter': [1000, 2000, 100]}, scoring='f1'),
   'best_model': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=12345),
   'best_score_train': 0.7551114973464736,
   'best_score_valid': 0.7545

Видим, что наилучшее качество метрики F1 получено при использовании модели логистической регрессии, сбалансированную по классам и количеством итераций, равным 1000, а также с использованием векторизатора CountVectorizer (мешок слов).

## Выводы

Необходимая модель найдена, метрика качества F1, полученная на тестовой выборке, равна 0.763 и удовлетворяет условию нашей задачи. Лучшая модель основанна на алгоритме LogisticRegression с использованием балансировки классов и количеством итераций, равным 1000, а также с использованием векторизатора CountVectorizer (мешок слов).

## Дальнейшие действия

Есть ли пути для улучшения результата? Да, есть.

Можно сделать следующее:
1. Провести векторизацию методом BERT. Это потребует серьёзных вычислительных мощностей.
2. Использовать RNN (LSTM).